# Tidying up our data - Part 1

# Learning objectives

- Manipulate deeply nested json data and transform them into structured data ready to be loaded into a Data Warehouse.

## Loading our data from S3

In [0]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/YOUTUBE/songs.json"

In [0]:
df = spark.read.format('json').load(filepath, multiline=True)

## Tidying up

---

We have multiple issues with our data.  **It does not look like "tidy data" at all.**  
First, we have rows within rows...
And second, most of the data resides in deeply nested structure within the column items...

We will fix the former, then handle the latter in the next notebook.

### 1. Fixing the rows
Remember the `.explode` method? [documentation](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.functions.explode).  
What `.explode(...)` does, it "Returns a new row for each element in the given array or map." We will use a lot of it here!

If you remember properly, that's exactly the kind of structures we have in the schema of our DataFrame for the `items` column.

1. Print out the schema of `df`

In [0]:
df.printSchema()

root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- contentDetails: struct (nullable = true)
 |    |    |    |-- caption: string (nullable = true)
 |    |    |    |-- contentRating: struct (nullable = true)
 |    |    |    |    |-- ytRating: string (nullable = true)
 |    |    |    |-- definition: string (nullable = true)
 |    |    |    |-- dimension: string (nullable = true)
 |    |    |    |-- duration: string (nullable = true)
 |    |    |    |-- licensedContent: boolean (nullable = true)
 |    |    |    |-- projection: string (nullable = true)
 |    |    |    |-- regionRestriction: struct (nullable = true)
 |    |    |    |    |-- allowed: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |-- blocked: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |-- etag: string (nullable = t

2. Import the PySpark SQL functions following usual convention

In [0]:
from pyspark.sql import functions as F

3. Use `.explode(...)` on the `items` column and count the number of results

In [0]:
df.withColumn('items', F.explode('items')).select('items').count()



Out[18]: 3907

If you got 3907 rows, you've made it, congrats! :)  
We will use this as our new working DataFrame:
- just do the same thing, but this time save the exploded dataset into a variable named `items_df`
- don't forget to give a proper alias to your newly compute column: `items`
- at the end, as a sanity check, make sure we have the right amount of columns in our new DataFrame

4. Follow previous instructions

In [0]:
items_df = df.withColumn('items', F.explode('items'))
items_df.select('items').count()


Out[19]: 3907

We're making progress, we now have one row per result (e.g. song)!

But each song is a deeply nested structure... We will take care of this in the following notebook.

5. Show the first 5 rows of the exploded dataset

In [0]:
items_df.limit(5).toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:122: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field items. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


,etag,items,kind,pageInfo
0,U0fncx_GV9jD5SKQr15LMvwuPcs,"((false, (None,), sd, 2d, PT3M33S, True, recta...",youtube#videoListResponse,"(38, 38)"
1,U0fncx_GV9jD5SKQr15LMvwuPcs,"((false, (None,), hd, 2d, PT7M46S, False, rect...",youtube#videoListResponse,"(38, 38)"
2,U0fncx_GV9jD5SKQr15LMvwuPcs,"((false, (None,), sd, 2d, PT3M7S, False, recta...",youtube#videoListResponse,"(38, 38)"
3,U0fncx_GV9jD5SKQr15LMvwuPcs,"((false, (None,), hd, 2d, PT3M43S, False, rect...",youtube#videoListResponse,"(38, 38)"
4,U0fncx_GV9jD5SKQr15LMvwuPcs,"((false, (None,), hd, 2d, PT5M, False, rectang...",youtube#videoListResponse,"(38, 38)"


In [0]:
items_df.select("items").limit(5).display()

items
"List(List(false, List(null), hd, 2d, PT7M46S, false, rectangular, List(null, List(DK))), m3DnhzTEw9ABiqzBvdasfk5Av_8, we5gzZq5Avg, youtube#video, List(10, UCson549gpvRhPnJ3Whs5onA, LongWayToDream, null, null, Air Conditionné EP, none, List(Air Conditionné EP, Julian Jeweil - Air Conditionné), 2012-03-17T08:34:30Z, List(mnml, minimal, electronic, music), List(List(90, https://i.ytimg.com/vi/we5gzZq5Avg/default.jpg, 120), List(360, https://i.ytimg.com/vi/we5gzZq5Avg/hqdefault.jpg, 480), List(720, https://i.ytimg.com/vi/we5gzZq5Avg/maxresdefault.jpg, 1280), List(180, https://i.ytimg.com/vi/we5gzZq5Avg/mqdefault.jpg, 320), List(480, https://i.ytimg.com/vi/we5gzZq5Avg/sddefault.jpg, 640)), Julian Jeweil - Air Conditionné), List(2, 3, 0, 124, 13409), List(true, youtube, false, public, true, processed), List(List(/m/04rlf, /m/04rlf), List(https://en.wikipedia.org/wiki/Music)))"
"List(List(false, List(null), sd, 2d, PT3M7S, false, rectangular, null), zyzs7STAR3NG-_pZe-0nGkbKoqg, 49esza4eiK4, youtube#video, List(10, UCcHYZ8Ez4gG_2bHEuBL8IfQ, Downtown Records, null, null, myspace.com/etjusticepourtous (Downtown / Ed Banger) Nominated for MTV VMA video of the year From the album 'Cross', none, List(myspace.com/etjusticepourtous (Downtown / Ed Banger) Nominated for MTV VMA video of the year From the album 'Cross', Justice - D.A.N.C.E), 2007-09-08T02:02:07Z, List(Justice, D.A.N.C.E, downtown), List(List(90, https://i.ytimg.com/vi/49esza4eiK4/default.jpg, 120), List(360, https://i.ytimg.com/vi/49esza4eiK4/hqdefault.jpg, 480), null, List(180, https://i.ytimg.com/vi/49esza4eiK4/mqdefault.jpg, 320), null), Justice - D.A.N.C.E), List(3168, 780, 0, 25540, 10106655), List(true, youtube, false, public, true, processed), List(List(/m/04rlf, /m/02lkt, /m/04rlf, /m/02lkt), List(https://en.wikipedia.org/wiki/Electronic_music, https://en.wikipedia.org/wiki/Music)))"
"List(List(false, List(null), hd, 2d, PT3M43S, false, rectangular, List(null, List(DK))), hX2C15F6fdO5A-stUFMU5Az2PvI, BoO6LfR7ca0, youtube#video, List(22, UCQ0wLCF7u23gZKJkHFs1Tpg, Music Is Our Drug, null, null, ♫ Music Is Our Drug - Spotify Playlist: https://goo.gl/rJvM5l ►Follow us on Spotify: https://goo.gl/HqOgkN Torture from the new Gramatik album - The Age Of Reason! Gramatik: https://soundcloud.com/gramatik https://www.facebook.com/gramatik Image: http://www.allwalls.net/wallpapers/2012/09/Women-Graffiti-Stadium-Bracelets-1800x2880.jpg (If any producer or label has an issue with any of the uploads please get in contact with me on YouTube and I will delete it as soon as possible (this includes artists of the images used)., none, List(♫ Music Is Our Drug - Spotify Playlist: https://goo.gl/rJvM5l ►Follow us on Spotify: https://goo.gl/HqOgkN Torture from the new Gramatik album - The Age Of Reason! Gramatik: https://soundcloud.com/gramatik https://www.facebook.com/gramatik Image: http://www.allwalls.net/wallpapers/2012/09/Women-Graffiti-Stadium-Bracelets-1800x2880.jpg (If any producer or label has an issue with any of the uploads please get in contact with me on YouTube and I will delete it as soon as possible (this includes artists of the images used)., Gramatik - Torture (feat. Eric Krasno)), 2014-01-24T12:52:38Z, List(Gramatik, Torture, The Age Of Reason, 2014, Music Is Our Drug, guitar, Eric Krisno), List(List(90, https://i.ytimg.com/vi/BoO6LfR7ca0/default.jpg, 120), List(360, https://i.ytimg.com/vi/BoO6LfR7ca0/hqdefault.jpg, 480), List(720, https://i.ytimg.com/vi/BoO6LfR7ca0/maxresdefault.jpg, 1280), List(180, https://i.ytimg.com/vi/BoO6LfR7ca0/mqdefault.jpg, 320), List(480, https://i.ytimg.com/vi/BoO6LfR7ca0/sddefault.jpg, 640)), Gramatik - Torture (feat. Eric Krasno)), List(6, 0, 0, 255, 29153), List(true, youtube, false, public, true, processed), List(List(/m/04rlf, /m/04rlf), List(https://en.wikipedia.org/wiki/Music)))"
"List(List(false, List(null), hd, 2d, PT5M, false, rectangular, List(null, List(DK))), rYHoV38PLpMbRuX_zhGTVBKNotw, DaH4W1rY9us, youtube#video, List(10, U

In [0]:
items_df.select("items").limit(5).show()

+--------------------+
|               items|
+--------------------+
|{{false, {null}, ...|
|{{false, {null}, ...|
|{{false, {null}, ...|
|{{false, {null}, ...|
|{{false, {null}, ...|
+--------------------+



## Wrap-up

You learned how to use `.explode(...)` to split arrays values into their own rows! 🎉